In [4]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession

# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

## Tabelas

In [5]:
contrato_sdf = spark.table("dev.dw_redeok_model.tb_d_contract")
customer_sdf= spark.table("dev.dw_redeok_model.tb_d_customer")

In [6]:
contrato_sdf.show(truncate=False)
contrato_sdf.count()

+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+--------+-------------------+-------------------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+-----------------------+------------------+--------------------------+---------+-----------+------------------------------------+
|uuid                                |uuid_customer                       |uuid_partner                        |uuid_wallet                         |uuid_business_model                 |status  |start_date         |end_date           |status_reason|consumption_value|access_fee|financial_fee|due_day|payment_frequency|max_ticket_value_in_month|data_source_insert|created_at             |data_source_update|updated_at                |uuid_user|entity_type|uuid_partner_contract               |
+-------------

61168

In [7]:
# Contar quantas chaves únicas existem em cada tabela
print("UUIDs Customers únicos em contrato_sdf:", contrato_sdf.select("uuid_customer").distinct().count())

print("UUIDs únicos em contrato_sdf:", contrato_sdf.select("uuid").distinct().count())

UUIDs Customers únicos em contrato_sdf: 57213
UUIDs únicos em contrato_sdf: 61168


In [8]:
# Agrupar por uuid_customer, contar e filtrar os que têm mais de um contrato
clientes_multiplos_contratos = contrato_sdf.groupBy("uuid_customer").count() \
    .filter(col("count") > 1) \
    .orderBy(col("count"), ascending=False)

# Exibir o resultado
clientes_multiplos_contratos.show(40, truncate=False)

+------------------------------------+-----+
|uuid_customer                       |count|
+------------------------------------+-----+
|9a2dde7c-04e6-4639-a24d-bfc7c39ce93c|275  |
|83a93536-475f-49dc-96dd-2d7081eb9b0d|84   |
|f41ebd28-9111-4fae-8dbf-51e3a43a61ce|19   |
|b034bd72-70bd-4251-9a8d-b08656b25508|17   |
|935c08da-bb9e-4cc9-8536-65492aad220e|14   |
|85c3d580-f467-4499-a818-d852791e9400|11   |
|c765a82a-072c-41bb-a253-66998011162d|10   |
|337bce7c-088c-457f-a2fe-570a6c9d2792|9    |
|08dd6f9a-5f31-46b7-a061-09bad0f5bade|9    |
|65255387-2b06-4cb6-91ec-7cc20262067e|8    |
|41a0ee2b-dd6c-43b7-8430-5adf1fdc523d|8    |
|014e0784-d1ea-40af-8d38-96493d33e779|8    |
|b93ce734-5c69-4fd5-90c2-368a9d08cbf5|7    |
|c0f90c1e-d032-4d20-8f91-d3755670cc87|7    |
|5a151ad9-d864-4ed3-b006-251a79094a62|6    |
|c6b515cb-87d1-4153-9d32-aa1d1f092285|6    |
|ef141ca9-5bb1-4ae8-81f2-3f03005b1700|6    |
|dda0965c-ba5a-4ed5-af98-d3c4e610b080|6    |
|97a0a465-7959-40c1-8784-ea1fc692ddca|5    |
|651b4178-

In [9]:
total_clientes_multiplos_contratos = clientes_multiplos_contratos.count()
print(f"Total de clientes com mais de um contrato: {total_clientes_multiplos_contratos}")

Total de clientes com mais de um contrato: 3086


In [10]:
uuid_especifico = "85c3d580-f467-4499-a818-d852791e9400"  # Substitua pelo UUID desejado

resultado_sdf = contrato_sdf.filter(contrato_sdf.uuid_customer == uuid_especifico)

# Mostrar resultado sem truncar

filtered_sdf_x = resultado_sdf.select("uuid_customer", "uuid", "status", "start_date", "end_date")

filtered_sdf_x.show(truncate=False)

+------------------------------------+------------------------------------+--------+-------------------+-------------------+
|uuid_customer                       |uuid                                |status  |start_date         |end_date           |
+------------------------------------+------------------------------------+--------+-------------------+-------------------+
|85c3d580-f467-4499-a818-d852791e9400|44612226-991c-40f1-8cec-ebcd29ced9bc|FINISHED|2010-11-26 02:00:00|2019-06-13 00:00:00|
|85c3d580-f467-4499-a818-d852791e9400|aff9e5e6-0463-4a25-9752-137797a22a24|FINISHED|2013-03-05 03:00:00|2016-07-18 00:00:00|
|85c3d580-f467-4499-a818-d852791e9400|4c523343-5d80-435b-80f3-4a2776eaed24|FINISHED|2014-01-28 02:00:00|2019-06-18 00:00:00|
|85c3d580-f467-4499-a818-d852791e9400|d73c1892-0dc1-4ea2-9612-515d33072dd8|SIGNED  |2012-11-12 02:00:00|NULL               |
|85c3d580-f467-4499-a818-d852791e9400|bb257f35-a3bb-4f1a-8905-c054ff01702d|FINISHED|2012-12-12 02:00:00|2015-02-25 03:00:00|


In [11]:
# Contar a quantidade de cada status
status_counts = contrato_sdf.groupBy(col("status")).count()

# Exibir o resultado
status_counts.show()

+-------------------+-----+
|             status|count|
+-------------------+-----+
|           APPROVED|   15|
|             SIGNED|19114|
|               NULL|  203|
|     IN_NEGOTIATION|   97|
|           FINISHED|41644|
|CANCELED_BY_PARTNER|   95|
+-------------------+-----+



In [12]:
# 1. Contar contratos onde end_date < start_date
contratos_invalidos = contrato_sdf.filter(col("end_date") < col("start_date")).count()

# 2. Contar contratos onde start_date é nulo e status é "SIGNED"
contratos_start_nulo_signed = contrato_sdf.filter((col("start_date").isNull()) & (col("status") == "SIGNED")).count()

# 3. Contar contratos com valores incomuns para end_date
# 3. Contar contratos com valores incomuns para end_date que começam com "0201"
contratos_valores_incomuns = contrato_sdf.filter(col("end_date").startswith("0201")).count()

# Exibir resultados
contratos_invalidos, contratos_start_nulo_signed, contratos_valores_incomuns

(111, 12, 49)

## Apenas  clientes com 1 Contrato

In [13]:
# Contar contratos por cliente
contratos_por_cliente = contrato_sdf.groupBy("uuid_customer").count()

# Filtrar apenas clientes com exatamente 1 contrato
clientes_com_um_contrato = contratos_por_cliente.filter(F.col("count") == 1)

# Juntar com a tabela original para obter todos os detalhes
sdf_filtrado = contrato_sdf.join(clientes_com_um_contrato, on="uuid_customer", how="inner")

# Exibir resultado
sdf_filtrado.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+-------------------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+--------------------+------------------+--------------------+---------+-----------+---------------------+-----+
|       uuid_customer|                uuid|        uuid_partner|         uuid_wallet| uuid_business_model|  status|         start_date|           end_date|status_reason|consumption_value|access_fee|financial_fee|due_day|payment_frequency|max_ticket_value_in_month|data_source_insert|          created_at|data_source_update|          updated_at|uuid_user|entity_type|uuid_partner_contract|count|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+-------------------+-------------+-----------------+----------+------------

In [14]:
sdf_filtrado.count()

54127

In [15]:
# Contar a quantidade de cada status
status_counts = sdf_filtrado.groupBy(col("status")).count()

# Exibir o resultado
status_counts.show()

+-------------------+-----+
|             status|count|
+-------------------+-----+
|           APPROVED|    9|
|             SIGNED|17080|
|               NULL|  196|
|     IN_NEGOTIATION|   72|
|           FINISHED|36691|
|CANCELED_BY_PARTNER|   79|
+-------------------+-----+



In [46]:
from pyspark.sql.functions import col

# Filtrar apenas os status "FINISHED"
status_counts = sdf_filtrado.filter(col("status") == "FINISHED").groupBy(col("status")).count()

# Exibir o resultado
status_counts.show()


+--------+-----+
|  status|count|
+--------+-----+
|FINISHED|36691|
+--------+-----+



In [16]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

# Definir a data de hoje
data_hoje = F.current_date()

# Criar coluna de tempo de permanência em anos
filtered_sdf = sdf_filtrado.withColumn(
    "end_date", F.when(F.col("end_date").isNull(), data_hoje).otherwise(F.col("end_date"))
).withColumn(
    "tempo_permanencia_dias", F.datediff(F.col("end_date"), F.col("start_date"))
).withColumn(
    "tempo_permanencia_anos", (F.col("tempo_permanencia_dias") / 365.25)  # Convertendo para anos
)

# Estatísticas básicas para tempo de permanência em anos
filtered_sdf.select("tempo_permanencia_anos").describe().show()

# Contas mais antigas
filtered_sdf.orderBy("start_date").show(truncate=False)

# Tempo médio de permanência em anos
tempo_medio_anos = filtered_sdf.select(F.avg("tempo_permanencia_anos")).collect()[0][0]
print(f"Tempo médio de permanência: {tempo_medio_anos:.2f} anos")


+-------+----------------------+
|summary|tempo_permanencia_anos|
+-------+----------------------+
|  count|                 53969|
|   mean|     4.496035819686011|
| stddev|     54.52408863684551|
|    min|    -1817.535934291581|
|    max|     28.62422997946612|
+-------+----------------------+

+------------------------------------+------------------------------------+------------------------------------+-----------+------------------------------------+-------------------+----------+-------------------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+--------------------------+------------------+--------------------------+------------------------------------+-----------+------------------------------------+-----+----------------------+----------------------+
|uuid_customer                       |uuid                                |uuid_partner                        |uuid_wallet|uuid_business_model       

In [17]:
from pyspark.sql.functions import col, when, datediff, current_date

# Definir a data de hoje
data_hoje = current_date()

filtered_sdf_2 = sdf_filtrado.withColumn(
    "end_date", F.when(F.col("end_date").isNull(), data_hoje).otherwise(F.col("end_date"))
).withColumn(
    "tempo_permanencia_dias", F.datediff(F.col("end_date"), F.col("start_date"))
).withColumn(
    "tempo_permanencia_anos", (F.col("tempo_permanencia_dias") / 365.25)  # Convertendo para anos
)

# Filtrar corretamente:
filtered_sdf_2 = filtered_sdf_2.filter(
    (~col("status").isin(["IN_NEGOTIATION", "CANCELED_BY_PARTNER", "APPROVED","SIGNED"])) & 
    (col("start_date").isNotNull()) & 
    (col("tempo_permanencia_dias") >= 0)  # Removendo quem tem tempo de permanência negativo
)

# Estatísticas básicas
filtered_sdf_2.describe("tempo_permanencia_anos").show()

# Contas mais antigas
contas_mais_antigas = filtered_sdf_2.orderBy("start_date").show(truncate=False)


# Tempo médio de permanência
tempo_medio = filtered_sdf_2.select(F.avg("tempo_permanencia_anos")).collect()[0][0]
print(f"Tempo médio de permanência: {tempo_medio:.2f} anos")

+-------+----------------------+
|summary|tempo_permanencia_anos|
+-------+----------------------+
|  count|                 36622|
|   mean|    5.3342720912475325|
| stddev|      4.55245332160149|
|    min|                   0.0|
|    max|    27.247091033538673|
+-------+----------------------+

+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+--------+-------------------+-------------------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+-----------------------+------------------+-------------------------+---------+-----------+------------------------------------+-----+----------------------+----------------------+
|uuid_customer                       |uuid                                |uuid_partner                        |uuid_wallet                         |uuid_busi

In [18]:
# Contar a quantidade de cada status
status_counts = filtered_sdf_2.groupBy(col("status")).count()

# Exibir o resultado
status_counts.show()

+--------+-----+
|  status|count|
+--------+-----+
|FINISHED|36622|
+--------+-----+



In [19]:
import plotly.graph_objects as go
import pandas as pd

# Converter para Pandas para visualização
df_permanencia = filtered_sdf_2.select("tempo_permanencia_anos", "status").toPandas()

# Definir as faixas de tempo de permanência em anos
bins = [0, 1, 2, 3, 4, 5, 6, 8, 10, df_permanencia["tempo_permanencia_anos"].max()]
labels = [
    "0-1 ano",
    "1-2 anos",
    "2-3 anos",
    "3-4 anos",
    "4-5 anos",
    "5-6 anos",
    "6-8 anos",
    "8-10 anos",
    "10+ anos"
]

# Criar coluna categórica com os intervalos
df_permanencia["faixa_permanencia"] = pd.cut(df_permanencia["tempo_permanencia_anos"], bins=bins, labels=labels, right=True)

# Contar a frequência de cada status por faixa
df_faixas_status = df_permanencia.groupby(["faixa_permanencia", "status"]).size().unstack(fill_value=0)

# Criar gráfico de barras agrupadas (lado a lado)
fig = go.Figure()

# Adicionar cada status como uma barra separada
for status in df_faixas_status.columns:
    fig.add_trace(go.Bar(
        x=df_faixas_status.index,
        y=df_faixas_status[status],
        name=status
    ))

# Configurar layout do gráfico
fig.update_layout(
    title="Distribuição do Tempo de Permanência dos Contratos Finalizados por Faixa (anos)",
    xaxis_title="Faixa de Tempo de Permanência (anos)",
    yaxis_title="Frequência",
    barmode="group"  # Barras lado a lado
)

# Exibir gráfico
fig.show()


C:\Users\gusta\AppData\Local\Temp\ipykernel_23524\1525092354.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_faixas_status = df_permanencia.groupby(["faixa_permanencia", "status"]).size().unstack(fill_value=0)


In [20]:
# Criar boxplot comparando o tempo de permanência entre contratos ativos (SIGNED) e finalizados (FINISHED)
fig_ativos_vs_finalizados = px.box(
    df_permanencia,
    x="status",
    y="tempo_permanencia_anos",
    title="Comparação do Tempo de Permanência: Contratos Finalizados",
    labels={"tempo_permanencia_anos": "Tempo de Permanência (anos)", "status": "Status do Contrato"},
    color="status"
)

# Exibir gráfico
fig_ativos_vs_finalizados.show()

In [25]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Converter para Pandas
df_permanencia = filtered_sdf_2.select("uuid_customer","start_date", "end_date", "tempo_permanencia_anos", "status").toPandas()

# Converter datas para datetime
df_permanencia["start_date"] = pd.to_datetime(df_permanencia["start_date"])
df_permanencia["end_date"] = pd.to_datetime(df_permanencia["end_date"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["start_date"].dt.year

# Criar coluna indicando se o contrato foi finalizado (Churn)
df_permanencia["churn"] = df_permanencia["status"].apply(lambda x: 1 if x == "FINISHED" else 0)

# Calcular taxa de churn por ano
df_churn = df_permanencia.groupby("ano_inicio")["churn"].mean().reset_index()

# Criar gráfico de evolução do churn por ano


# Criar histograma do tempo de permanência dos contratos finalizados
fig_hist_churn = px.histogram(
    df_permanencia[df_permanencia["status"] == "FINISHED"],
    x="tempo_permanencia_anos",
    nbins=30,
    title="Distribuição do Tempo de Permanência dos Contratos Finalizados",
    labels={"tempo_permanencia_anos": "Tempo de Permanência (anos)"},
    marginal="box"  # Adiciona um boxplot para melhor visualização
)

# Exibir gráficos
fig_hist_churn.show()

In [38]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = filtered_sdf_2.select("start_date", "tempo_permanencia_anos").toPandas()

# Converter start_date para datetime
df_permanencia["start_date"] = pd.to_datetime(df_permanencia["start_date"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["start_date"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular estatísticas por ano
estatisticas_por_ano = df_permanencia.groupby("ano_inicio")["tempo_permanencia_anos"].agg(["median", "mean"]).reset_index()

# Calcular o tempo máximo possível para cada ano (tempo entre ano de início e hoje)
estatisticas_por_ano["tempo_maximo_possivel"] = ano_atual - estatisticas_por_ano["ano_inicio"]

# Calcular a diferença entre o tempo máximo e a mediana
estatisticas_por_ano["diferenca"] = estatisticas_por_ano["tempo_maximo_possivel"] - estatisticas_por_ano["median"]

# Identificar o primeiro ano onde a diferença for menor que um limite (exemplo: 2 anos)
ano_limite_vies = estatisticas_por_ano.loc[estatisticas_por_ano["diferenca"] < 2, "ano_inicio"].min()

# Criar gráfico interativo
fig = go.Figure()

# Linha da mediana do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["median"],
    mode="lines+markers",
    name="Mediana do Tempo de Permanência",
    line=dict(color="blue")
))

# Linha da média do tempo de permanência por ano
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["mean"],
    mode="lines+markers",
    name="Média do Tempo de Permanência",
    line=dict(color="green")
))

# Linha do tempo máximo possível
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["tempo_maximo_possivel"],
    mode="lines",
    name="Tempo Máximo Possível",
    line=dict(color="red", dash="dash")
))



# Layout do gráfico
fig.update_layout(
    title="Detecção do Viés no Tempo de Permanência",
    xaxis_title="Ano de Início do Contrato",
    yaxis_title="Tempo de Permanência (anos)",
    showlegend=True
)

# Exibir gráfico
fig.show()

# Exibir o ano identificado como enviesado
print(f"O primeiro ano onde os contratos finalizados são afetados pelo tempo transcorrido é: {ano_limite_vies}")


O primeiro ano onde os contratos finalizados são afetados pelo tempo transcorrido é: 2023


In [47]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = filtered_sdf_2.select("start_date", "tempo_permanencia_anos").toPandas()

# Converter start_date para datetime
df_permanencia["start_date"] = pd.to_datetime(df_permanencia["start_date"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["start_date"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular o tempo máximo possível para cada ano
df_permanencia["tempo_maximo_possivel"] = ano_atual - df_permanencia["ano_inicio"]

# Criar Boxplot
fig = px.box(
    df_permanencia,
    x="ano_inicio",  # Ano do início do contrato no eixo X
    y="tempo_permanencia_anos",  # Tempo de permanência no eixo Y
    points="all",  # Mostra todos os pontos para visualizar outliers
    title="Boxplot da Distribuição do Tempo de Permanência por Ano de Início do Contrato",
    labels={"ano_inicio": "Ano de Início do Contrato", "tempo_permanencia_anos": "Tempo de Permanência (anos)"},
    color="ano_inicio"  # Diferencia os anos por cor
)

# Adicionar linha guia do tempo máximo possível
fig.add_trace(
    go.Scatter(
        x=df_permanencia["ano_inicio"].unique(),
        y=(ano_atual - df_permanencia["ano_inicio"].unique()),  # Tempo máximo possível
        mode="lines",
        name="Tempo Máximo Possível",
        line=dict(color="red", dash="dash")
    )
)



# Exibir gráfico
fig.show()

# Exibir o ano identificado como enviesado
print(f"O primeiro ano onde os contratos finalizados são afetados pelo tempo transcorrido é: {ano_limite_vies}")


O primeiro ano onde os contratos finalizados são afetados pelo tempo transcorrido é: 1997


In [44]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = filtered_sdf_2.select("start_date", "tempo_permanencia_anos").toPandas()

# Converter start_date para datetime
df_permanencia["start_date"] = pd.to_datetime(df_permanencia["start_date"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["start_date"].dt.year

# Criar categoria de períodos
bins = [1995, 2005, 2015, 2025]
labels = ["1995-2005", "2005-2015", "2015-2025"]
df_permanencia["periodo"] = pd.cut(df_permanencia["ano_inicio"], bins=bins, labels=labels, right=False)

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular o tempo máximo possível para cada ano (tempo entre ano de início e hoje)
df_permanencia["tempo_maximo_possivel"] = ano_atual - df_permanencia["ano_inicio"]

# Criar Boxplot separado por período com melhor visualização
fig = px.box(
    df_permanencia,
    x="ano_inicio",  # Ano do início do contrato no eixo X
    y="tempo_permanencia_anos",  # Tempo de permanência no eixo Y
    color="periodo",  # Diferenciar os períodos por cores
    title="Boxplot da Distribuição do Tempo de Permanência por Período",
    labels={"ano_inicio": "Ano de Início do Contrato", "tempo_permanencia_anos": "Tempo de Permanência (anos)"},
    points="suspectedoutliers",  # Mostrar apenas outliers
    notched=True  # Adicionar entalhe nas caixas para melhor leitura
)

# Adicionar linha guia do tempo máximo possível
fig.add_trace(
    go.Scatter(
        x=df_permanencia["ano_inicio"].unique(),
        y=(ano_atual - df_permanencia["ano_inicio"].unique()),  # Tempo máximo possível
        mode="lines",
        name="Tempo Máximo Possível",
        line=dict(color="red", dash="dash", width=3)  # Linha mais espessa
    )
)

# Melhorar layout para melhor espaçamento e aumentar tamanho
fig.update_layout(
    xaxis_title="Ano de Início do Contrato",
    yaxis_title="Tempo de Permanência (anos)",
    showlegend=True,
    boxmode="group",  # Separar melhor os períodos
    width=1600,  # Aumentar a largura do gráfico
    height=500   # Aumentar a altura do gráfico
)

# Exibir gráfico
fig.show()


In [45]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

# Converter para Pandas
df_permanencia = filtered_sdf_2.select("start_date", "tempo_permanencia_anos").toPandas()

# Converter start_date para datetime
df_permanencia["start_date"] = pd.to_datetime(df_permanencia["start_date"])

# Criar coluna com o ano do contrato
df_permanencia["ano_inicio"] = df_permanencia["start_date"].dt.year

# Obter o ano atual
ano_atual = datetime.today().year

# Calcular estatísticas por ano
estatisticas_por_ano = df_permanencia.groupby("ano_inicio")["tempo_permanencia_anos"].agg(["median", "mean", "std", "count"]).reset_index()

# Calcular o tempo máximo possível para cada ano (tempo entre ano de início e hoje)
estatisticas_por_ano["tempo_maximo_possivel"] = ano_atual - estatisticas_por_ano["ano_inicio"]

# Calcular a razão entre a mediana e o tempo máximo possível
estatisticas_por_ano["proporcao_mediana"] = estatisticas_por_ano["median"] / estatisticas_por_ano["tempo_maximo_possivel"]

# Determinar o primeiro ano onde a proporção da mediana fica muito próxima de 1 (indicando viés)
ano_limite_vies = estatisticas_por_ano.loc[estatisticas_por_ano["proporcao_mediana"] > 0.85, "ano_inicio"].min()

# Criar gráfico para visualizar essa relação
fig = go.Figure()

# Linha da proporção da mediana
fig.add_trace(go.Scatter(
    x=estatisticas_por_ano["ano_inicio"],
    y=estatisticas_por_ano["proporcao_mediana"],
    mode="lines+markers",
    name="Proporção Mediana / Tempo Máximo",
    line=dict(color="blue")
))

# Adicionar linha horizontal indicando o limite de viés (exemplo: 0.85 como referência)
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=min(estatisticas_por_ano["ano_inicio"]), x1=max(estatisticas_por_ano["ano_inicio"]),
        y0=0.85, y1=0.85,
        line=dict(color="red", dash="dash"),
    )
)

# Adicionar linha vertical no ano identificado como enviesado
if not pd.isna(ano_limite_vies):
    fig.add_shape(
        go.layout.Shape(
            type="line",
            x0=ano_limite_vies, x1=ano_limite_vies,
            y0=0, y1=1,
            line=dict(color="black", dash="dot"),
        )
    )

    # Anotação do ano limite
    fig.add_annotation(
        x=ano_limite_vies,
        y=0.9,
        text=f"Ano Limite: {ano_limite_vies}",
        showarrow=True,
        arrowhead=2,
        font=dict(size=12, color="black"),
        bgcolor="white",
        bordercolor="black",
    )

# Ajuste do layout
fig.update_layout(
    title="Proporção da Mediana do Tempo de Permanência em Relação ao Tempo Máximo Possível",
    xaxis_title="Ano de Início do Contrato",
    yaxis_title="Proporção Mediana / Tempo Máximo",
    showlegend=True
)

# Exibir gráfico
fig.show()

# Exibir o ano identificado como enviesado
print(f"O primeiro ano onde os contratos começam a ser fortemente limitados pelo tempo transcorrido é: {ano_limite_vies}")


O primeiro ano onde os contratos começam a ser fortemente limitados pelo tempo transcorrido é: 1997
